# **Minerando Dados - A maior comunidade de Data Science do Brasil**
www.minerandodados.com.br

## **Análise de Sentimentos usando Machine Learning**

* Criando modelos para análise de sentimentos de tweets
* Teste com Modelo usando tag de negações
* Teste com Modelo usando Bigrams

In [60]:
from nltk import word_tokenize
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

**Ler arquivo de dados e conta a quantidade de linhas**

In [61]:
import json

In [62]:
arquivo = open('tweets.txt', 'r')

In [63]:
def cria_lista(arquivo):
    lista = []
    
    linha = arquivo.readline().strip()
    while linha != "":
        if (linha != "\n"):
            tweet = json.loads(linha)
            created_at = tweet["created_at"]
            id_str = tweet["id_str"]
            text = tweet["text"]
            
            objeto = {"created_at":created_at, "id_str":id_str, "text":text}
            
            lista.append(objeto)
        linha = arquivo.readline()
        
    return lista

In [64]:
lista = cria_lista(arquivo)

In [65]:
print(lista)

[{'created_at': 'Tue Sep 10 13:52:02 +0000 2019', 'id_str': '1171421071708426240', 'text': 'ADEUS NETFLIX COM SEU CATALOGO DE FILMES LIXOSO'}, {'created_at': 'Tue Sep 10 13:52:04 +0000 2019', 'id_str': '1171421083322527744', 'text': 'Eu já tinha o Prime Video, alterei pra assinatura do Prime que vai sair mais barata e com muito mais benefícios e o… https://t.co/xkhWbqnt70'}, {'created_at': 'Tue Sep 10 13:52:04 +0000 2019', 'id_str': '1171421083620249601', 'text': 'RT @HLTCO: Sky Sports minimum cost: £40 per month\n\nBT Sport minimum cost: £30 pm\n\nPremier Sports cost: £11.99 pm\n\nAmazon Prime cost: £7.99…'}, {'created_at': 'Tue Sep 10 13:52:06 +0000 2019', 'id_str': '1171421088389062659', 'text': 'RT @bonjurliana: indo assinar o amazon prime https://t.co/wTWS36Wvoa'}, {'created_at': 'Tue Sep 10 13:52:08 +0000 2019', 'id_str': '1171421096631046144', 'text': 'Amazon prime por 10 conto com o prime vídeo, twitch prime e o serviço de música era tudo que eu precisava'}, {'created_at': 'Tue

In [66]:
lista[2]

{'created_at': 'Tue Sep 10 13:52:04 +0000 2019',
 'id_str': '1171421083620249601',
 'text': 'RT @HLTCO: Sky Sports minimum cost: £40 per month\n\nBT Sport minimum cost: £30 pm\n\nPremier Sports cost: £11.99 pm\n\nAmazon Prime cost: £7.99…'}

In [83]:
def cria_arquivo_csv(lista):
    arquivo = open('tweets.csv', 'w', encoding='utf-8')
    lista_chave = []
    
    #arquivo.write('id,')
        
    for chave in lista[0]:
        lista_chave.append(chave)
            
    for pos in range(len(lista_chave)):
        if pos == len(lista_chave)-1:
            arquivo.write(lista_chave[pos] + '\n')
        else:
            arquivo.write(lista_chave[pos] + ',')
    
    for dic in lista:
        for pos in range(len(lista_chave)):
            aux = dic[lista_chave[pos]]
            if lista_chave[pos] == 'text':
                aux = '"' + dic[lista_chave[pos]] + '"'
                
            if pos == len(lista_chave)-1:
                arquivo.write(aux + '\n')
            else:
                arquivo.write(aux + ',')
        
    arquivo.close()

In [84]:
cria_arquivo_csv(lista)

In [85]:
dataset = pd.read_csv('tweets.csv', encoding='utf-8')

In [86]:
type(dataset)

pandas.core.frame.DataFrame

**Exibe as 50 primeiras lihas de tweets**

In [87]:
dataset.head(10)

,created_at,id_str,text
0,Tue Sep 10 13:52:02 +0000 2019,1171421071708426240,ADEUS NETFLIX COM SEU CATALOGO DE FILMES LIXOSO
1,Tue Sep 10 13:52:04 +0000 2019,1171421083322527744,"Eu já tinha o Prime Video, alterei pra assinat..."
2,Tue Sep 10 13:52:04 +0000 2019,1171421083620249601,RT @HLTCO: Sky Sports minimum cost: £40 per mo...
3,Tue Sep 10 13:52:06 +0000 2019,1171421088389062659,RT @bonjurliana: indo assinar o amazon prime h...
4,Tue Sep 10 13:52:08 +0000 2019,1171421096631046144,"Amazon prime por 10 conto com o prime vídeo, t..."
5,Tue Sep 10 13:52:08 +0000 2019,1171421099722252288,RT @milapellicer: Sobre o Amazon Prime no BR: ...
6,Tue Sep 10 13:52:09 +0000 2019,1171421101424988160,RT @TOEI_PR: 君のヒーローは誰だ！？\r\n\r\n50～90年代の懐かしの特撮...
7,Tue Sep 10 13:52:09 +0000 2019,1171421102100373505,RT @HLTCO: Sky Sports minimum cost: £40 per mo...
8,Tue Sep 10 13:52:10 +0000 2019,1171421105069920256,Amazon Prime chegou pra acabar com aquele disc...
9,Tue Sep 10 13:52:11 +0000 2019,1171421111936069633,Obrigado capitalismo opressor


**Conta a quantidade de linhas de tweets neutros, positivos e negativos**

In [88]:
dataset[dataset.Classificacao=='Neutro'].count()

AttributeError: 'DataFrame' object has no attribute 'Classificacao'

In [ ]:
dataset[dataset.Classificacao=='Positivo'].count()

In [ ]:
dataset[dataset.Classificacao=='Negativo'].count()

In [89]:
%matplotlib inline
dataset.Classificacao.value_counts().plot(kind='bar')

AttributeError: 'DataFrame' object has no attribute 'Classificacao'

In [ ]:
dataset.count()

## Pre-Processamento dos Dados

* Remove linhas duplicadas na base de dados
    - Problema na coleta dos dados.
* Remove Stopwords
* Faz Stemming nos dados
* Remove caracteres indesejados como links, pontuação etc.

In [91]:
dataset.drop_duplicates(['text'], inplace=True)

In [93]:
dataset.text.count()

9

## ** Separando tweets e suas Classes**

In [94]:
tweets = dataset['text']
#classes = dataset['Classificacao']

** Instala bibliotecas e baixa a base de dados**

In [95]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping stemmers\rslp.zip.


True

**Funções de Pre-processamento de dados**

In [96]:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [97]:
def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

In [98]:
def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

** Entenda como funciona cada função**

In [99]:
RemoveStopWords('Eu não gosto do partido, e também não votaria novamente nesse governante!')

'Eu gosto partido, votaria novamente nesse governante!'

In [100]:
Stemming('Eu não gosto do partido, e também não votaria novamente nesse governante!')

'eu não gost do partido, e também não vot nov ness governante!'

In [101]:
Limpeza_dados('Assita aqui o video do Governador falando sobre a CEMIG https://www.uol.com.br :) ;)')

'assita aqui o video do governador falando sobre a cemig   '

** Aplica as 3 funções de Pre-processamento nos dados**

In [102]:
def Preprocessing(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [stemmer.stem(i) for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# Aplica a função em todos os dados:
tweets = [Preprocessing(i) for i in tweets]

In [103]:
Preprocessing('Eu não gosto do partido, e também não votaria novamente nesse governante. Assita o video aqui https:// :)')

'gost partido, vot nov ness govern assit vide aqu'

In [111]:
for i in dataset:
    print(i)

created_at
id_str
text


**Visualize os dados e veja como ficou após o pré-processamento**

In [ ]:
tweets[:50]

## Criando o modelo

**Instancia o objeto que faz a vetorização dos dados de texto**

In [104]:
vectorizer = CountVectorizer(analyzer="word")

**Aplica o vetorizador nos dados de texto**

In [105]:
freq_tweets = vectorizer.fit_transform(tweets)
type(freq_tweets)

scipy.sparse.csr.csr_matrix

In [106]:
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

NameError: name 'classes' is not defined

**Formato (Linhas, Colunas) da matriz**

In [107]:
freq_tweets.shape

(9, 74)

**Matriz**

In [108]:
freq_tweets.A

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1, 2, 0,
        0, 1, 1, 0, 1, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0,

** Testando o modelo com algumas instâncias simples**

In [ ]:
# defina instâncias de teste dentro de uma lista
testes = ['Esse governo está no início, vamos ver o que vai dar',
          'Estou muito feliz com o governo de Minas esse ano',
          'O estado de Minas Gerais decretou calamidade financeira!!!',
          'A segurança desse país está deixando a desejar',
          'O governador de Minas é mais uma vez do PT']

**Aplica a função de Pré-processamento nos dados**

In [ ]:
testes = [Preprocessing(i) for i in testes]

In [ ]:
# Transforma os dados de teste em vetores de palavras.
freq_testes = vectorizer.transform(testes)

In [ ]:
# Fazendo a classificação com o modelo treinado.
for t, c in zip (testes,modelo.predict(freq_testes)):
    print (t +", "+ c)

In [ ]:
# Probabilidades de cada classe
print (modelo.classes_)
modelo.predict_proba(freq_testes).round(2)

## ** Função de Tags de Negações**

* Acrescenta uma tag _NEG encontrada após um 'não'.
* Objetivo é dar mais peso para o modelo identificar uma inversão de sentimento da frase.
* Exemplos: 
    - Eu gosto de cachorros, positivo.
    - Eu **não** gosto de cachorros, negativo.

In [112]:
def marque_negacao(texto):
    negacoes = ['não','not']
    negacao_detectada = False
    resultado = []
    palavras = texto.split()
    for p in palavras:
        p = p.lower()
        if negacao_detectada == True:
            p = p + '_NEG'
        if p in negacoes:
            negacao_detectada = True
        resultado.append(p)
    return (" ".join(resultado))

**Exemplos de utilização da tag de negações**

In [ ]:
marque_negacao('Eu gosto do partido, votaria novamente nesse governante!')

In [ ]:
marque_negacao('Eu Não gosto do partido, e também não votaria novamente nesse governante!')

## **Criando modelos com Pipelines**

* Pipelines são interessantes para reduzir código e automatizar fluxos

In [113]:
from sklearn.pipeline import Pipeline

In [114]:
pipeline_simples = Pipeline([
  ('counts', CountVectorizer()),
  ('classifier', MultinomialNB())
])

* Pipeline que atribui tag de negacoes nas palavras

In [115]:
pipeline_negacoes = Pipeline([
  ('counts', CountVectorizer(tokenizer=lambda text: marque_negacao(text))),
  ('classifier', MultinomialNB())
])

In [116]:
pipeline_simples.fit(tweets,classes)

NameError: name 'classes' is not defined

In [117]:
pipeline_simples.steps

[('counts',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                  lowercase=True, max_df=1.0, max_features=None, min_df=1,
                  ngram_range=(1, 1), preprocessor=None, stop_words=None,
                  strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                  tokenizer=None, vocabulary=None)),
 ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

* Gera o modelo de negações

In [118]:
pipeline_negacoes.fit(tweets,classes)

NameError: name 'classes' is not defined

* Etapas do pipeline

In [119]:
pipeline_negacoes.steps

[('counts',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                  lowercase=True, max_df=1.0, max_features=None, min_df=1,
                  ngram_range=(1, 1), preprocessor=None, stop_words=None,
                  strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                  tokenizer=<function <lambda> at 0x04C2E1E0>, vocabulary=None)),
 ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]

## Validando os Modelos com Validação Cruzada

* Fazendo o cross validation do modelo

In [120]:
resultados = cross_val_predict(pipeline_simples, tweets, classes, cv=10)

NameError: name 'classes' is not defined

* Medindo a acurácia média do modelo

In [121]:
metrics.accuracy_score(classes,resultados)

NameError: name 'classes' is not defined

* Medidas de validação do modelo

In [122]:
sentimento=['Positivo','Negativo','Neutro']
print (metrics.classification_report(classes,resultados,sentimento))

NameError: name 'classes' is not defined

* Matriz de confusão

In [123]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

NameError: name 'classes' is not defined

## **Modelo com a Tag de Negações**

In [124]:
resultados = cross_val_predict(pipeline_negacoes, tweets, classes, cv=10)

NameError: name 'classes' is not defined

* Medindo a acurácia média do modelo

In [ ]:
metrics.accuracy_score(classes,resultados)

In [ ]:
sentimento=['Positivo','Negativo','Neutro']
print (metrics.classification_report(classes,resultados,sentimento))

* Matriz de confusão

In [ ]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

## ** Avaliando modelo com Bigrams**

In [ ]:
'eu gosto', 'gosto do' , 'do brasil'

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,2))
freq_tweets = vectorizer.fit_transform(tweets)
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

In [ ]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [ ]:
metrics.accuracy_score(classes,resultados)

In [ ]:
sentimento=['Positivo','Negativo','Neutro']
print (metrics.classification_report(classes,resultados,sentimento))

In [ ]:
print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))

## ** Considerações Finais**

* Considere aumentar a quantidade de dados de treino.

* Pela sua simplicidade o Naive Bayes pode ser usado perfeitamente como um algoritmo de Baseline.

* Considere alterar os parâmetros do algoritmo.

**www.minerandodados.com.br** - *A maior comunidade de Data Science do Brasil*